## Notebook to determine important carriers for the STST and EXP networks

In [40]:
# imports
import pypsa
import pandas as pd

from utils import nodal_balance
from utils import get_condense_sum, carrier_renaming, carrier_renaming_reverse
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

# stst and exp dataframes
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [2]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [7]:
print(stst)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 3
 - Line: 323
 - Link: 8375
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


In [8]:
print(exp)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 2
 - Line: 323
 - Link: 9192
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


### Carrier differences

In [11]:
# same carriers for generators
set(exp.generators.carrier.unique().tolist()).difference(stst.generators.carrier.unique().tolist())
stst.generators.carrier.unique().tolist()

['offwind-ac',
 'onwind',
 'solar',
 'ror',
 'offwind-dc',
 'gas',
 'residential rural solar thermal',
 'services rural solar thermal',
 'residential urban decentral solar thermal',
 'services urban decentral solar thermal',
 'urban central solar thermal',
 'oil',
 'solar rooftop']

In [15]:
# EXP has two additional links: 'H2 pipeline', 'H2 pipeline retrofitted'
set(exp.links.carrier.unique().tolist()).difference(stst.links.carrier.unique().tolist())
exp.links.carrier.unique().tolist()

['DC',
 'OCGT',
 'H2 Electrolysis',
 'H2 Fuel Cell',
 'H2 pipeline retrofitted',
 'H2 pipeline',
 'battery charger',
 'battery discharger',
 'Sabatier',
 'SMR CC',
 'SMR',
 'BEV charger',
 'V2G',
 'residential rural ground heat pump',
 'residential rural water tanks charger',
 'residential rural water tanks discharger',
 'residential rural resistive heater',
 'residential rural gas boiler',
 'services rural ground heat pump',
 'services rural water tanks charger',
 'services rural water tanks discharger',
 'services rural resistive heater',
 'services rural gas boiler',
 'residential urban decentral air heat pump',
 'residential urban decentral water tanks charger',
 'residential urban decentral water tanks discharger',
 'residential urban decentral resistive heater',
 'residential urban decentral gas boiler',
 'services urban decentral air heat pump',
 'services urban decentral water tanks charger',
 'services urban decentral water tanks discharger',
 'services urban decentral resisti

In [18]:
# same carriers for storage units
set(stst.storage_units.carrier.unique().tolist()).difference(exp.storage_units.carrier.unique().tolist())
stst.storage_units.carrier.unique().tolist()

['hydro', 'PHS']

In [20]:
# overall number of carriers:
# STST: 68
# EXP: 70
len(stst.storage_units.carrier.unique().tolist() + stst.links.carrier.unique().tolist() + stst.generators.carrier.unique().tolist())

68

### Relevant carriers per technology
- classify in electricity (gen / con), hydrogen (gen, con) and heat (gen, con)
- determine by share of generation / consumption

In [23]:
# electricity

# overall electricity generation in TWh per technology
carrier = ["AC", "low voltage"]
nb = nodal_balance(stst, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)
# calc sum and convert to TWh
(nb[nb > 0].sum() / 1000).sort_values()

carrier
AC                            0.000009
BEV charger                   0.000000
DAC                           0.000000
DC                            0.000029
H2 Electrolysis               0.000000
H2 Fuel Cell                  1.409125
OCGT                          3.031344
PHS                          46.621278
V2G                          99.732769
agriculture electricity       0.000000
battery charger               0.000000
battery discharger          388.014856
electricity                   0.000000
home battery charger          0.000000
home battery discharger       0.130558
hydro                       348.187357
industry electricity          0.000000
offwind-ac                  182.048413
offwind-dc                  609.327531
onwind                     4556.001892
ror                         138.641227
solar                      3255.956798
solar rooftop               328.158738
gas CHP                     139.465576
gas CHP CC                    0.007028
biomass CHP CC   

In [65]:
pos_gen = (nb[nb > 0].sum() / 1000).sort_values(ascending=False)[(nb[nb > 0].sum() / 1000).sort_values(ascending=False) > 0.001]
pos_gen.index = [carrier_renaming_reverse.get(n, n) for n in pos_gen.index]
pos_gen.sort_values(ascending=False)

onwind                                4595.271018
solar                                 2339.819359
offwind-dc                             968.839829
hydro                                  348.196304
solar rooftop                          263.249380
offwind-ac                             144.159188
ror                                    139.380125
battery discharger                     108.545008
urban central solid biomass CHP CC      81.040705
V2G                                     64.558623
urban central gas CHP                   47.400430
PHS                                     41.703688
urban central solid biomass CHP         27.095899
OCGT                                     2.957557
home battery discharger                  0.007775
dtype: float64

In [61]:
(df_stst_ons[[c + "_gen" for c in pos_gen.index]].sum() / 1000) / (df_stst_ons[[c + "_gen" for c in pos_gen.index]].sum() / 1000).sum()

onwind_gen                                4.461034e-01
solar_gen                                 3.188086e-01
offwind-dc_gen                            5.966305e-02
hydro_gen                                 3.409345e-02
solar rooftop_gen                         3.213226e-02
offwind-ac_gen                            1.782664e-02
ror_gen                                   1.357516e-02
battery discharger_gen                    3.799385e-02
urban central solid biomass CHP CC_gen    1.031478e-02
V2G_gen                                   9.768269e-03
urban central gas CHP_gen                 1.365622e-02
PHS_gen                                   5.749362e-03
urban central solid biomass CHP_gen       6.304349e-07
OCGT_gen                                  2.971311e-04
home battery discharger_gen               1.709538e-05
dtype: float64

In [52]:
# electricity

# overall electricity generation in TWh per technology
carrier = ["AC", "low voltage"]
nb = nodal_balance(exp, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)
# calc sum and convert to TWh
(nb[nb > 0].sum() / 1000).sort_values()

carrier
industry electricity       0.000000e+00
water tanks charger        0.000000e+00
heat pump                  0.000000e+00
gas boiler                 0.000000e+00
resistive heater           0.000000e+00
water tanks discharger     0.000000e+00
home battery charger       0.000000e+00
electricity                0.000000e+00
solar thermal              0.000000e+00
agriculture electricity    0.000000e+00
BEV charger                0.000000e+00
H2 Electrolysis            0.000000e+00
battery charger            0.000000e+00
AC                         5.688746e-07
DC                         2.994737e-05
home battery discharger    7.775209e-03
OCGT                       2.957557e+00
biomass CHP                2.709590e+01
PHS                        4.170369e+01
gas CHP                    4.740043e+01
V2G                        6.455862e+01
biomass CHP CC             8.104071e+01
battery discharger         1.085450e+02
ror                        1.393801e+02
offwind-ac                 1.441

In [63]:
pos_gen = (nb[nb < 0].sum() / 1000).sort_values(ascending=False)[(nb[nb < 0].sum() / 1000).sort_values(ascending=False) < -0.001]
pos_gen.index = [carrier_renaming_reverse.get(n, n) for n in pos_gen.index]
pos_gen.sort_values(ascending=True)

H2 Electrolysis                  -3558.103289
electricity                      -1720.767554
industry electricity             -1702.520000
BEV charger                       -946.278666
heat pump                         -787.444014
urban central resistive heater    -239.795041
battery charger                   -113.068737
PHS                                -59.385843
agriculture electricity            -44.861105
home battery charger                -0.015978
dtype: float64

In [75]:
(df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum() / 1000).sort_values(ascending=False)

electricity distribution grid_gen         5048.708774
onwind_gen                                4556.006981
solar_gen                                 3255.958746
BEV charger_gen                            890.745672
offwind-dc_gen                             609.332350
battery charger_gen                        396.028538
battery discharger_gen                     388.027177
hydro_gen                                  348.192743
solar rooftop_gen                          328.163361
offwind-ac_gen                             182.061598
urban central gas CHP_gen                  139.469472
ror_gen                                    138.641710
urban central solid biomass CHP CC_gen     105.343787
V2G_gen                                     99.762294
PHS_gen                                     58.717624
OCGT_gen                                     3.034568
H2 Fuel Cell_gen                             1.416061
home battery charger_gen                     0.178194
home battery discharger_gen 

In [76]:
(df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum() / 1000).sort_values(ascending=False)

electricity distribution grid_gen         5113.896804
onwind_gen                                4595.279875
solar_gen                                 2339.825054
offwind-dc_gen                             968.844260
BEV charger_gen                            851.662131
hydro_gen                                  348.200933
solar rooftop_gen                          263.256595
offwind-ac_gen                             144.170812
ror_gen                                    139.380591
battery charger_gen                        110.822269
battery discharger_gen                     108.583215
urban central solid biomass CHP CC_gen      81.051047
V2G_gen                                     64.587106
PHS_gen                                     53.051560
urban central gas CHP_gen                   47.403712
urban central solid biomass CHP_gen         27.102189
OCGT_gen                                     2.959376
home battery charger_gen                     0.116080
home battery discharger_gen 

In [70]:
# investigate in carriers with most generation for both networks
# without grid ("electricity distribution grid", "DC", "H2 pipeline", "H2 pipeline retrofitted")
# storage technologies:

carriers_el_gen = [
    "electricity distribution grid",
    "onwind",
    "solar",
    "BEV charger",
    "offwind-dc",
    "ror",
    "hydro",
    "solar rooftop",
    "offwind-ac",
    "battery charger",
    "battery discharger",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "OCGT",
    "home battery charger",
    "home battery discharger",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DC"
]

carriers_el_con = [
    "PHS",
    "electricity distribution grid",
    "H2 Electrolysis",
    "BEV charger",
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central resistive heater",
    "battery charger",
    "battery discharger",
    "V2G",
    "residential urban decentral resistive heater",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "services rural resistive heater",
    "home battery charger",
    "home battery discharger",
    "DC",
    "DAC",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell",
    "Sabatier",
    "Fischer-Tropsch"
]

carriers_heat_gen1 = [
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central gas boiler",
    "urban central resistive heater",
    "residential urban decentral gas boiler",
    "urban central water tanks charger",
    "urban central water tanks discharger",
    "residential urban decentral resistive heater",
    "services urban decentral gas boiler",
    "urban central solar thermal",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "residential rural gas boiler",
    "services rural resistive heater",
    "services rural gas boiler",
    "services rural water tanks charger",
    "residential rural water tanks charger",
    "services rural water tanks discharger",
    "residential rural water tanks discharger",
    "services urban decentral water tanks charger",
    "residential urban decentral water tanks charger",
    "services urban decentral water tanks discharger",
    "residential urban decentral water tanks discharger",
    "residential urban decentral solar thermal",
    "services urban decentral solar thermal",
    "residential rural solar thermal",
    "services rural solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
"urban central water tanks discharger",
"services rural water tanks charger",
"residential rural water tanks charger",
"services rural water tanks discharger",
"residential rural water tanks discharger",
"services urban decentral water tanks charger",
"residential urban decentral water tanks charger",
"services urban decentral water tanks discharger",
"residential urban decentral water tanks discharger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

In [71]:
# select carriers which make most sense

# ordered according to (df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum() + df_exp_ons[[c + "_gen" for c in c_el_gen_s]].sum()).sort_values(ascending=False)
# overall less than 50 TWh: urban central gas CHP CC, fuel cell, home battery discharger, OCGT, urban central solid biomass CHP,
c_el_gen_s = [
    "onwind",
    "solar",
    "offwind-dc",
    "hydro",
    "solar rooftop",
    "battery discharger",
    "offwind-ac",
    "ror",
    "urban central gas CHP",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# overall less than 50 TWh: home battery charger
# ordered according to (df_stst_ons[[c + "_con_el" for c in c_el_con_s]].sum() + df_exp_ons[[c + "_con_el" for c in c_el_con_s]].sum()).sort_values(ascending=False)
c_el_con_s = [
    "H2 Electrolysis",
    "BEV charger",
    "battery charger",
    "urban central air heat pump",
    "urban central resistive heater",
    "PHS",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell"
    "Sabatier",
    "Fischer-Tropsch"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# use urban central gas boiler (max generation)
# use urban central water tanks charger (max generation)
# use urban central solar thermal (max generation)

carriers_heat_gen1 = [
    "urban central air heat pump"
    "urban central gas boiler"
    "urban central resistive heater"
    "urban central water tanks discharger"
    "urban central solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

In [79]:
n = stst
all_c = n.generators.carrier.unique().tolist() + n.links.carrier.unique().tolist() + n.storage_units.carrier.unique().tolist()

# carriers not in any list:
set(all_c).difference(carriers_el_gen + carriers_el_con + carriers_h2_gen + carriers_h2_con + carriers_heat_gen1 + carriers_heat_con)

{'Sabatier',
 'biogas to gas',
 'gas',
 'gas for industry',
 'gas for industry CC',
 'oil',
 'process emissions',
 'process emissions CC',
 'residential rural gas boiler',
 'residential rural solar thermal',
 'residential rural water tanks charger',
 'residential rural water tanks discharger',
 'residential urban decentral gas boiler',
 'residential urban decentral solar thermal',
 'residential urban decentral water tanks charger',
 'residential urban decentral water tanks discharger',
 'services rural gas boiler',
 'services rural solar thermal',
 'services rural water tanks charger',
 'services rural water tanks discharger',
 'services urban decentral gas boiler',
 'services urban decentral solar thermal',
 'services urban decentral water tanks charger',
 'services urban decentral water tanks discharger',
 'solid biomass for industry',
 'solid biomass for industry CC',
 'urban central gas boiler',
 'urban central solar thermal',
 'urban central water tanks discharger'}

In [ ]:
# Check second and third output of links
# what is with DAC?

In [ ]:
n.links[n.links.carrier == "DAC"]

In [81]:
# exclude technologies with less than 0.5 % of total generation or consumption in both scenarios
stst_car = df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 51*1e3].index
exp_car   = df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 46*1e3].index
set(stst_car).intersection(exp_car)

{'DC_gen',
 'H2 Fuel Cell_gen',
 'OCGT_gen',
 'home battery charger_gen',
 'home battery discharger_gen',
 'urban central gas CHP CC_gen',
 'urban central solid biomass CHP_gen'}